In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000

100% 5.20G/5.20G [02:45<00:00, 35.1MB/s]
100% 5.20G/5.20G [02:45<00:00, 33.7MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/skin-cancer-mnist-ham10000.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.applications import MobileNetV2

In [ ]:
base_model=MobileNetV2(include_top=False, weights="imagenet", input_shape=(224,224,3))

9406464/9406464 [==============================] - 1s 0us/step


In [ ]:
model = Sequential()
model.add(base_model)
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,3)))
model.add(Dropout(0.40))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.40))
model.add(Dense(7,activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 conv2d (Conv2D)             (None, 5, 5, 64)          737344    
                                                                 
 max_pooling2d (MaxPooling2D  (None, 2, 1, 64)         0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 2, 1, 64)          0         
                                                                 
 flatten (Flatten)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 128)               1

In [ ]:
data=pd.read_csv("HAM10000_metadata.csv")
data['image_full_name']=data['image_id']+'.jpg'
X=data[['image_full_name','dx','lesion_id']]

In [ ]:
X.sample(5)

,image_full_name,dx,lesion_id
6013,ISIC_0027267.jpg,nv,HAM_0002870
7803,ISIC_0033688.jpg,nv,HAM_0004424
9027,ISIC_0025776.jpg,nv,HAM_0003785
6179,ISIC_0032326.jpg,nv,HAM_0000280
6055,ISIC_0031010.jpg,nv,HAM_0002910


In [ ]:
from sklearn.model_selection import train_test_split
if 'dx' in X.columns:
  Y=X.pop('dx').to_frame()
  X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.17,random_state=42)
  X_train,X_val,y_train,y_val  = train_test_split(X_train, y_train, test_size=0.17, random_state=42)
else:
  print("'dx' column does not exist in X.")

In [ ]:
train=pd.concat([X_train,y_train],axis=1)
val=pd.concat([X_val,y_val],axis=1)
test=pd.concat([X_test,y_test],axis=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(val['dx'])
name_as_indexes_train=encoder.transform(val['dx'])
val['label']=name_as_indexes_train

In [ ]:
encoder = LabelEncoder()
encoder.fit(test['dx'])
name_as_indexes_test=encoder.transform(test['dx'])
test['label']=name_as_indexes_test

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator
train_generator = ImageDataGenerator(rescale = 1./255,
                                     rotation_range = 10,
                                     zoom_range = 0.1,
                                     )

In [ ]:
X_train

,image_full_name,lesion_id
3736,ISIC_0027575.jpg,HAM_0002758
147,ISIC_0033646.jpg,HAM_0003743
4723,ISIC_0026025.jpg,HAM_0002396
1374,ISIC_0028127.jpg,HAM_0005911
269,ISIC_0030124.jpg,HAM_0004807
...,...,...
8504,ISIC_0028212.jpg,HAM_0006266
4006,ISIC_0029167.jpg,HAM_0000213
40,ISIC_0031650.jpg,HAM_0003847
2456,ISIC_0032057.jpg,HAM_0004862


In [ ]:
train_data = train_generator.flow_from_dataframe(dataframe=train,x_col="image_full_name",y_col="dx",
                                                 batch_size=32,directory="/content/ham10000_images_part_1",
                                                 shuffle=True, class_mode="categorical",target_size=(224,224))

Found 3452 validated image filenames belonging to 7 classes.


/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 3446 invalid image filename(s) in x_col="image_full_name". These filename(s) will be ignored.
  warnings.warn(


In [ ]:
test_generator=ImageDataGenerator(rescale=1./255,rotation_range=10,
                                  zoom_range=0.1, width_shift_range=0.0,height_shift_range=0.02)

In [ ]:
test_data=test_generator.flow_from_dataframe(dataframe=test,x_col="image_full_name",y_col="dx",
                                                directory="/content/ham10000_images_part_1",
                                             shuffle=False,batch_size=1,class_mode=None,target_size=(224,224))

Found 836 validated image filenames.


/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 867 invalid image filename(s) in x_col="image_full_name". These filename(s) will be ignored.
  warnings.warn(


In [ ]:
val_data=test_generator.flow_from_dataframe(dataframe=val,x_col="image_full_name",y_col="dx",
                                                directory="/content/ham10000_images_part_1",
                                             batch_size=64,shuffle=False,class_mode="categorical",target_size=(224,224))

Found 712 validated image filenames belonging to 7 classes.


/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 702 invalid image filename(s) in x_col="image_full_name". These filename(s) will be ignored.
  warnings.warn(


In [ ]:
from keras.callbacks import ReduceLROnPlateau
learning_control=ReduceLROnPlateau(monitor='val_acc',patience=3,verbose=1,factor=.5,min_lr=0.0001)

In [ ]:
from keras.optimizers.adam import optimizer
from keras.optimizers import Adam,SGD
sgd=optimizers.SGD(lr=0.01,clipvalue=0.5)
model.compile(optimizer=sgd,loss="categorical_crossentropy",metrics=["accuracy"])

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
print(learning_control)

In [ ]:
history = model.fit_generator(generator=train_data,
                            steps_per_epoch=train_data.samples//train_data.batch_size,
                            validation_data=val_data,
                            verbose=1,
                            validation_steps=val_data.samples//val_data.batch_size,
                            epochs=25,callbacks=[learning_control])

<ipython-input-38-e32234db7242>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(generator=train_data,


Epoch 1/8
107/107 [==============================] - ETA: 0s - loss: 0.3819 - accuracy: 0.8731

107/107 [==============================] - 820s 8s/step - loss: 0.3819 - accuracy: 0.8731 - val_loss: 1.3814 - val_accuracy: 0.5540 - lr: 0.0100
Epoch 2/8
107/107 [==============================] - ETA: 0s - loss: 0.3751 - accuracy: 0.8740

107/107 [==============================] - 870s 8s/step - loss: 0.3751 - accuracy: 0.8740 - val_loss: 1.2663 - val_accuracy: 0.7088 - lr: 0.0100
Epoch 3/8
107/107 [==============================] - ETA: 0s - loss: 0.3437 - accuracy: 0.8833

107/107 [==============================] - 810s 8s/step - loss: 0.3437 - accuracy: 0.8833 - val_loss: 1.1850 - val_accuracy: 0.6364 - lr: 0.0100
Epoch 4/8
107/107 [==============================] - ETA: 0s - loss: 0.3374 - accuracy: 0.8813

107/107 [==============================] - 782s 7s/step - loss: 0.3374 - accuracy: 0.8813 - val_loss: 1.1396 - val_accuracy: 0.6705 - lr: 0.0100
Epoch 5/8
107/107 [==============================] - ETA: 0s - loss: 0.3145 - accuracy: 0.8906

107/107 [==============================] - 803s 7s/step - loss: 0.3145 - accuracy: 0.8906 - val_loss: 1.0490 - val_accuracy: 0.7131 - lr: 0.0100
Epoch 6/8
107/107 [==============================] - ETA: 0s - loss: 0.2623 - accuracy: 0.9096

107/107 [==============================] - 806s 8s/step - loss: 0.2623 - accuracy: 0.9096 - val_loss: 1.4561 - val_accuracy: 0.7386 - lr: 0.0100
Epoch 7/8
107/107 [==============================] - ETA: 0s - loss: 0.2158 - accuracy: 0.9228

107/107 [==============================] - 763s 7s/step - loss: 0.2158 - accuracy: 0.9228 - val_loss: 1.4071 - val_accuracy: 0.7628 - lr: 0.0100
Epoch 8/8
107/107 [==============================] - ETA: 0s - loss: 0.2010 - accuracy: 0.9333

107/107 [==============================] - 744s 7s/step - loss: 0.2010 - accuracy: 0.9333 - val_loss: 2.2716 - val_accuracy: 0.7244 - lr: 0.0100


In [ ]:
model.save('skincancer.h5')

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','val'],loc='upper left')
plt.show()


In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','val'],loc='upper left')
plt.show()

In [ ]:
test_data.reset()
predictions = model.predict_generator(test_data,steps=test_data.samples/test_data.batch_size,verbose=1)
y_pred=np.argmax(predictions,axis=1)


In [ ]:
c=np.where(y_pred==name_as_indexes_test)

In [ ]:
np.count_nonzero(c)

In [ ]:
print("Test Accuracy :",(np.count_nonzero(c)/len(name_as_indexes_test))*100)

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(name_as_indexes_test, y_pred)
print(report)

In [ ]:
model.save("skincancermodel.h5")

In [ ]:
!pip install streamlit

In [ ]:
%%writefile app.py


In [ ]:
import streamlit as st
import tensorflow as tf

st.set_option('deprecation.showfileUploaderEncoding',False)
@st.cache(allow_output_mutation=True)
def load_model():
  model=tf.keras.models.load_model('skincancermodel')
  return model
model=load_model()
st.write("Health Care")
st.write("""
        #Skin Lesion Detector Web tool
        """
        )

file=st.file_uploader("Choose...", type=["jpg","png"])
import cv2
from PIL import Image,ImageOps
import numpy as np
def import_and_predict(image_data,model):
  size=(180,180)
  image=ImageOps.fit(image_data,size,Image.ANTIALIAS)
  img=np.asarray(image)
  img_reshape=img[np.newaxis,...]
  prediction=model.predict(img_reshape)

  return prediction

if file is None:
  st.text("Please upload an image file")
else:
  image=Image.open(file)
  st.image(image,use_column_width=True)
  predictions=import_and_predict(image,model)
  class_names=[]


